In [44]:
from lake import FrozenLakeEnvStop
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [45]:
game = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=0)

In [46]:
print(game.reset())
print(game.render())
print(game.step(0))
print(game.step(1))
print(game.step(1))
print(game.step(1))

{'state': 0, 'gave_feedback': False}

SFFF
FHFH
FFFH
HFFG
None
({'state': 0, 'gave_feedback': False}, 0.0, False, {'prob': 1.0})
({'state': 4, 'gave_feedback': False}, 0.0, False, {'prob': 1.0})
({'state': 8, 'gave_feedback': False}, 0.0, False, {'prob': 1.0})
({'state': 8, 'gave_feedback': True}, 0, False, {'prob': 1.0})


In [47]:
print(game.observation_space)
print(game.action_space)
# action are : 0 Left     1 Down      2  RIGHT     3 UP    

Discrete(16)
Discrete(4)


In [49]:
def run_expe(create_algo, game, verbose=True):
    
    n_ep = 1000    
    count_success = 0
    success_to_win = 5
    
    algo = create_algo()
    
    for ep in range(n_ep):
        
        next_state = None
        state = game.reset()
        done = False
        cumul_reward = 0
        num_steps = 0
        while not done:
            
            action = algo.choose_action_eps_greedy(state)
            next_state, reward, done, info = game.step(action)
            algo.optimize(state, action, next_state, reward)
            
            cumul_reward += reward
            state = next_state
            num_steps += 1
        
        #if last reward is 1, we consider that the environment is solved.
        # if the algo solve the algo N times in a row, it's okay ! (N = success_to_win)
        if reward == 1:
            count_success += 1
        else:
            count_success = 0
            
        if count_success > success_to_win:
            break
        if verbose:
            print("End of ep #{:05d} in {:03d} steps, cumul_reward = {}".format(ep,  num_steps, cumul_reward))
        
    return ep

In [57]:
def run_multiple_expe(create_algo, game, n_expe):
    
    mean_num_step = []
    for i in range(n_expe):
        mean_num_step.append(run_expe(create_algo, game, verbose=False))
        
    mean_num_step = np.array(mean_num_step)
    print("Number of episodes mean", np.mean(mean_num_step))
    print("Number of failure to solve env", np.count_nonzero(mean_num_step > (game.max_steps - 5)))
    
    episodes_of_non_failure = mean_num_step[np.where(mean_num_step<995)]
    print("Number of episodes when not failing", episodes_of_non_failure.mean())

In [52]:
class TabQLearning(object):
    def __init__(self, env_size, n_action, gamma, lr, expected_exploration_steps):
        
        self.n_action = n_action
        self.env_size = env_size
        self.q_tab = np.zeros((env_size, n_action))
        
        self.lr = lr
        self.gamma = gamma
        
        # Exploration parameters
        self.expected_exploration_steps = expected_exploration_steps
        self.n_step_eps = 0
        self.minimum_epsilon = 0.05
        self.epsilon_init = 1   
        self.current_eps = self.epsilon_init
    def choose_action_eps_greedy(self, s):
        
        if np.random.random() < self.current_eps:
            a = np.random.randint(self.n_action)
        else:
            s = s['state']
            a = np.argmax(self.q_tab[s])
            
        self.current_eps = max(self.minimum_epsilon, self.epsilon_init * np.exp(- 2.5 * self.n_step_eps / self.expected_exploration_steps))
        self.n_step_eps += 1
        
        return a
    
    def optimize(self, state, a, next_state, r):
        state, next_state = state['state'], next_state['state']
        self.q_tab[state, a] = (1-self.lr)*self.q_tab[state, a] + self.lr*(r + self.gamma*np.max(self.q_tab[next_state]))

In [53]:
class TabQLearningControlerFeedback(object):
    def __init__(self, env_size, n_action, gamma, lr, expected_exploration_steps, margin):
        
        self.n_action = n_action
        self.env_size = env_size
        self.q_tab = np.zeros((env_size, n_action))
        
        self.lr = lr
        self.gamma = gamma
        self.margin = margin
        
        # Exploration parameters
        self.expected_exploration_steps = expected_exploration_steps
        self.n_step_eps = 0
        self.minimum_epsilon = 0.05
        self.epsilon_init = 1   
        self.current_eps = self.epsilon_init
    def choose_action_eps_greedy(self, s):
        
        s = s['state']
        
        if np.random.random() < self.current_eps:
            a = np.random.randint(self.n_action)
        else:
            a = np.argmax(self.q_tab[s])
            
        self.current_eps = max(self.minimum_epsilon, self.epsilon_init * np.exp(- 2.5 * self.n_step_eps / self.expected_exploration_steps))
        self.n_step_eps += 1
        
        return a
    
    def optimize(self, state, action, next_state, r):
        
        state = state['state']

        gave_feedback = next_state['gave_feedback']        
        next_state = next_state['state']

        if gave_feedback:
            bad_action = action
            q_with_margin = [self.q_tab[state, a] - self.margin_loss(bad_action, a) for a in range(self.n_action)]
            self.q_tab[state, action] = (1 - self.lr) * self.q_tab[state, action] + self.lr * min(q_with_margin)
        else:
            self.q_tab[state, action] = (1 - self.lr) * self.q_tab[state, action] + self.lr * (r + self.gamma * np.max(self.q_tab[next_state]))
            
    def margin_loss(self, bad_action, a):
        if bad_action == a:
            return 0
        else:
            return self.margin
        
gamma = 0.99
expected_exploration_steps = 3000
margin=0.2

create_algo = lambda : TabQLearningControlerFeedback(16, 4, gamma=gamma,
                                                     lr=0.1, expected_exploration_steps=expected_exploration_steps, margin=margin)

game = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=0)
run_expe(create_algo, game, verbose=True)

End of ep #00000 in 088 steps, cumul_reward = 1.0
End of ep #00001 in 100 steps, cumul_reward = 0.0
End of ep #00002 in 100 steps, cumul_reward = 0.0
End of ep #00003 in 032 steps, cumul_reward = 1.0
End of ep #00004 in 100 steps, cumul_reward = 0.0
End of ep #00005 in 100 steps, cumul_reward = 0.0
End of ep #00006 in 100 steps, cumul_reward = 0.0
End of ep #00007 in 078 steps, cumul_reward = 1.0
End of ep #00008 in 092 steps, cumul_reward = 1.0
End of ep #00009 in 088 steps, cumul_reward = 1.0
End of ep #00010 in 023 steps, cumul_reward = 1.0
End of ep #00011 in 012 steps, cumul_reward = 1.0


12

Frozen-Lake Deterministe avec Feedback
-------------------

Q-Learning Tabulaire, pas de récompenses négatives

In [59]:
gamma = 0.99
expected_exploration_steps = 3000
margin= 0.2

algo = lambda : TabQLearning(16, 4, gamma=gamma, lr=0.1, expected_exploration_steps=expected_exploration_steps)
game = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=0)

run_multiple_expe(algo, game, n_expe=1000)

Number of episodes mean 55.937
Number of failure to solve env 51
Number of episodes when not failing 10.464360587002096


Q-Learning Tabulaire, récompenses négatives de l'environnement

In [60]:
gamma = 0.99
expected_exploration_steps = 3000
margin= 0.2

algo = lambda : TabQLearning(16, 4, gamma=gamma, lr=0.1, expected_exploration_steps=expected_exploration_steps)
game = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=-0.1)

run_multiple_expe(algo, game, n_expe=1000)

Number of episodes mean 48.94
Number of failure to solve env 47
Number of episodes when not failing 9.354166666666666


Q-Learning Tabulaire inclusion du Feedback, pas de récompenses négatives de l'environnement

In [61]:
gamma = 0.99
expected_exploration_steps = 3000
margin= 0.2

algo = lambda : TabQLearningControlerFeedback(16, 4, gamma=gamma, lr=0.1, expected_exploration_steps=expected_exploration_steps, margin=margin)
game = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=0)

run_multiple_expe(algo, game, n_expe=1000)

Number of episodes mean 32.568
Number of failure to solve env 27
Number of episodes when not failing 7.787692307692308


Frozen-Lake Non-Deterministe avec Feedback
-------------------

In [66]:
game = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=0)
print("deterministe", game.P)


game = FrozenLakeEnvStop(is_slippery=True, reward_when_falling=0)
print("non deterministic", game.P)

deterministe {0: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 4, 0.0, False)], 2: [(1.0, 1, 0.0, False)], 3: [(1.0, 0, 0.0, False)]}, 1: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 5, 0.0, True)], 2: [(1.0, 2, 0.0, False)], 3: [(1.0, 1, 0.0, False)]}, 2: {0: [(1.0, 1, 0.0, False)], 1: [(1.0, 6, 0.0, False)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 2, 0.0, False)]}, 3: {0: [(1.0, 2, 0.0, False)], 1: [(1.0, 7, 0.0, True)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 3, 0.0, False)]}, 4: {0: [(1.0, 4, 0.0, False)], 1: [(1.0, 8, 0.0, False)], 2: [(1.0, 5, 0.0, True)], 3: [(1.0, 0, 0.0, False)]}, 5: {0: [(1.0, 5, 0, True)], 1: [(1.0, 5, 0, True)], 2: [(1.0, 5, 0, True)], 3: [(1.0, 5, 0, True)]}, 6: {0: [(1.0, 5, 0.0, True)], 1: [(1.0, 10, 0.0, False)], 2: [(1.0, 7, 0.0, True)], 3: [(1.0, 2, 0.0, False)]}, 7: {0: [(1.0, 7, 0, True)], 1: [(1.0, 7, 0, True)], 2: [(1.0, 7, 0, True)], 3: [(1.0, 7, 0, True)]}, 8: {0: [(1.0, 8, 0.0, False)], 1: [(1.0, 12, 0.0, True)], 2: [(1.0, 9, 0.0, False)], 3: [(1.0, 4, 0.0, False)]

In [74]:
gamma = 0.99
expected_exploration_steps = 3000
margin=0.2

create_algo = lambda : TabQLearningControlerFeedback(16, 4, gamma=gamma,
                                                     lr=0.1, expected_exploration_steps=expected_exploration_steps, margin=margin)

game = FrozenLakeEnvStop(is_slippery=True, reward_when_falling=-0.1)

run_expe(create_algo, game, verbose=True)




SFFF
FHFH
FFFH
HFFG
None
End of ep #00000 in 069 steps, cumul_reward = 1.1102230246251565e-16
End of ep #00001 in 100 steps, cumul_reward = -1.4000000000000001
End of ep #00002 in 022 steps, cumul_reward = 0.8
End of ep #00003 in 100 steps, cumul_reward = -1.7000000000000004
End of ep #00004 in 049 steps, cumul_reward = -0.19999999999999996
End of ep #00005 in 088 steps, cumul_reward = -0.8000000000000005
End of ep #00006 in 044 steps, cumul_reward = 0.6
End of ep #00007 in 090 steps, cumul_reward = 0.20000000000000007
End of ep #00008 in 030 steps, cumul_reward = 0.7


9

This seems very easy to solve because of the controller stopping from doing stupid stuff, the explo is easy.

Q-Learning Tabulaire, pas de récompenses négatives

In [76]:
gamma = 0.99
expected_exploration_steps = 3000
margin= 0.2

algo = lambda : TabQLearning(16, 4, gamma=gamma, lr=0.1, expected_exploration_steps=expected_exploration_steps)
game = FrozenLakeEnvStop(is_slippery=True, reward_when_falling=0)

run_multiple_expe(algo, game, n_expe=1000)

Number of episodes mean 14.911
Number of failure to solve env 2
Number of episodes when not failing 12.938877755511022


Q-Learning Tabulaire, récompenses négatives de l'environnement

In [77]:
gamma = 0.99
expected_exploration_steps = 3000
margin= 0.2

algo = lambda : TabQLearning(16, 4, gamma=gamma, lr=0.1, expected_exploration_steps=expected_exploration_steps)
game = FrozenLakeEnvStop(is_slippery=True, reward_when_falling=-0.1)

run_multiple_expe(algo, game, n_expe=1000)

Number of episodes mean 23.509
Number of failure to solve env 22
Number of episodes when not failing 23.509


Q-Learning Tabulaire inclusion du Feedback, pas de récompenses négatives de l'environnement

In [ ]:
gamma = 0.99
expected_exploration_steps = 3000
margin= 0.2

algo = lambda : TabQLearningControlerFeedback(16, 4, gamma=gamma, lr=0.1, expected_exploration_steps=expected_exploration_steps, margin=margin)
game = FrozenLakeEnvStop(is_slippery=True, reward_when_falling=0)

run_multiple_expe(algo, game, n_expe=1000)

In [7]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [9]:
class QModel(nn.Module):
    def __init__(self, size_in, n_action):
        super().__init__()
        
        self.n_hidden = 10
        self.n_action = n_action
        self.input_size = size_in
        
        self.fc1 = nn.Linear(self.input_size, self.n_hidden)
        self.fc2 = nn.Linear(self.n_hidden, self.n_action)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [10]:
steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            qs = policy_net(state)
            return qs.max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [11]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [12]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 40
LR = 6e-3

env = gym.make('CartPole-v0').unwrapped

env_size = env.observation_space.shape[0]
n_actions = env.action_space.n

policy_net = QModel(size_in=env_size, n_action=n_actions).to(device)
target_net = QModel(size_in=env_size, n_action=n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters(), lr=LR)
memory = ReplayMemory(10000)

num_episodes = 1000
cumul_reward = 0
rew_list = []


for i_episode in range(num_episodes):
    # Initialize the environment and state
    state = torch.FloatTensor(env.reset()).unsqueeze(0)
    assert isinstance(state, torch.Tensor)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        next_state, reward, done, _ = env.step(action.item())
        
        cumul_reward += reward
        
        reward = torch.tensor([reward], device=device)
        next_state = torch.FloatTensor(next_state).unsqueeze(0)
        assert isinstance(next_state, torch.Tensor)
        
        if done:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            rew_list.append(cumul_reward)
            cumul_reward = 0
            break
    
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        print("End of ep #{}, cumul_reward : {}".format(i_episode, np.mean(rew_list)))
        rew_list = []

        target_net.load_state_dict(policy_net.state_dict())
    



End of ep #0, cumul_reward : 35.0


End of ep #40, cumul_reward : 42.175


End of ep #80, cumul_reward : 16.525


End of ep #120, cumul_reward : 25.6


End of ep #160, cumul_reward : 139.8


...

End of ep #800, cumul_reward : 171.325


End of ep #840, cumul_reward : 131.6


End of ep #880, cumul_reward : 253.675


End of ep #920, cumul_reward : 202.175


End of ep #960, cumul_reward : 168.625


In [24]:
num_episodes = 1000
cumul_reward = 0
rew_list = []

BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 30
LR = 3e-3

env = FrozenLakeEnvStop(is_slippery=False, reward_when_falling=-0.1)

n_actions = env.action_space.n
state_size = 1


policy_net = QModel(size_in=state_size, n_action=n_actions).to(device)
target_net = QModel(size_in=state_size, n_action=n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters(), lr=LR)
memory = ReplayMemory(10000)


for i_episode in range(num_episodes):
    # Initialize the environment and state
    state = env.reset()
    state = torch.FloatTensor([state]).unsqueeze(0)
    assert isinstance(state, torch.Tensor)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        next_state, reward, done, _ = env.step(action.item())
        
        cumul_reward += reward
        
        reward = torch.tensor([reward], device=device)
        next_state = torch.FloatTensor([next_state]).unsqueeze(0)
        assert isinstance(next_state, torch.Tensor)
        
        if done:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            rew_list.append(cumul_reward)
            cumul_reward = 0
            break
    
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        print("End of ep #{}, cumul_reward : {}".format(i_episode, np.mean(rew_list)))
        rew_list = []

        target_net.load_state_dict(policy_net.state_dict())

End of ep #0, cumul_reward : 0.0


End of ep #30, cumul_reward : -0.1233333333333333


End of ep #60, cumul_reward : 0.043333333333333314


End of ep #90, cumul_reward : 0.019999999999999976


End of ep #120, cumul_reward : 0.13000000000000017


...

End of ep #870, cumul_reward : -0.0033333333333333335


End of ep #900, cumul_reward : -0.04000000000000001


End of ep #930, cumul_reward : -0.07666666666666667


End of ep #960, cumul_reward : -0.22


End of ep #990, cumul_reward : -0.04000000000000001
